In [ ]:
!pip install -U tfx

     |████████████████████████████████| 2.4 MB 15.8 MB/s 
     |████████████████████████████████| 422 kB 79.8 MB/s 
     |████████████████████████████████| 135 kB 67.1 MB/s 
     |████████████████████████████████| 1.8 MB 55.8 MB/s 
     |████████████████████████████████| 9.9 MB 65.7 MB/s 
     |████████████████████████████████| 19.1 MB 471 kB/s 
     |████████████████████████████████| 40 kB 4.9 MB/s 
     |████████████████████████████████| 1.4 MB 62.6 MB/s 
     |████████████████████████████████| 1.7 MB 61.9 MB/s 
     |████████████████████████████████| 98 kB 6.0 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 42.8 MB/s 
     |████████████████████████████████| 1.7 MB 86.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.8 MB/s 
     |████████████████████████████████| 147 kB 85.9 MB/s 
     |████████████████████████████████| 205 kB 61.4 MB/s 
     |████████████████████████████████| 151 kB 71.4 MB/s 
     |██████████████

In [ ]:
import os
import pprint
import numpy as np
import tempfile
import urllib

import absl
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from google.protobuf.json_format import MessageToDict
from tfx.components import CsvExampleGen
from tfx.components.common_nodes.importer_node import ImporterNode
from typing import Dict, List, Text
from tfx.components import Evaluator, ExampleValidator, Pusher, SchemaGen, StatisticsGen, Transform, Tuner, Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2, trainer_pb2, tuner_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing, HyperParameters
from tfx.types import Channel


from tfx.extensions.google_cloud_ai_platform.trainer import component as gcp_trainer
from tfx.extensions.google_cloud_ai_platform.tuner import component as gcp_tuner
from tfx.extensions.google_cloud_ai_platform.pusher import component as gcp_pusher
from tfx.extensions.google_cloud_ai_platform.constants import ENABLE_VERTEX_KEY
from tfx.extensions.google_cloud_ai_platform.constants import VERTEX_REGION_KEY

#from tensorflow_cloud import CloudTuner

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.7.0
TFX version: 1.5.0


In [ ]:
# Create TFX required directories
!rm -rf data.*
!rm -rf *trainer.py
!sudo rm -rf tfx/*

! mkdir tfx
! mkdir tfx/pipelines
! mkdir tfx/metadata
! mkdir tfx/logs
! mkdir tfx/data
! mkdir tfx/serving_model

In [ ]:
!wget 'https://raw.githubusercontent.com/Biswajit7890/DataZeit_Coding_Challenge/main/SalaryDataCleaned.csv'


--2022-01-26 12:26:13--  https://raw.githubusercontent.com/Biswajit7890/DataZeit_Coding_Challenge/main/SalaryDataCleaned.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3628356 (3.5M) [text/plain]
Saving to: ‘SalaryDataCleaned.csv’

SalaryDataCleaned.c 100%[===================>]   3.46M  --.-KB/s    in 0.03s   

2022-01-26 12:26:13 (108 MB/s) - ‘SalaryDataCleaned.csv’ saved [3628356/3628356]



In [ ]:
df = pd.read_csv('/content/SalaryDataCleaned.csv')
df.to_csv("tfx/data/data_trans.csv", index=False, header=True)

In [ ]:
_tfx_root = os.path.join(os.getcwd(), 'tfx');        
_pipeline_root = os.path.join(_tfx_root, 'pipelines');      
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db');   
_log_root = os.path.join(_tfx_root, 'logs');
_model_root = os.path.join(_tfx_root, 'model');
_data_root = os.path.join(_tfx_root, 'data');
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_data_filepath = os.path.join(_data_root, "data_trans.csv")

_transform_module_file = 'transform_trainer.py'
_model_trainer_module_file = 'model.py'

In [ ]:
context = InteractiveContext(pipeline_root=_tfx_root)

In [ ]:
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 2
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/content/tfx/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:3596377,xor_checksum:1643199976,sum_checksum:1643199976"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1643199984950
        last_update_time_since_epoch: 1643199998772
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 3
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "/content/tfx/StatisticsGen/statistics/3"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [ ]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/content/tfx/SchemaGen/schema/4"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'SalaryGT50K',INT,required,,-
'age',INT,required,,-
'capital_gain',INT,required,,-
'capital_loss',INT,required,,-
'education',STRING,required,,'education'
'education_num',INT,required,,-
'fnlwgt',INT,required,,-
'hours_per_week',INT,required,,-
'marital_status',STRING,required,,'marital_status'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'marital_status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native_country',"' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Holand-Netherlands', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


In [ ]:
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],
                              schema=schema_gen.outputs['schema'])

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "/content/tfx/ExampleValidator/anomalies/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context.show(example_validator.outputs['anomalies'])

In [ ]:
import pandas as pd
train_df=pd.read_csv('/content/SalaryDataCleaned.csv')
train_df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,SalaryGT50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [ ]:
train_df.nunique()

age                  73
workclass             9
fnlwgt            21406
education            16
education_num        16
marital_status        7
occupation           15
relationship          6
race                  5
sex                   2
capital_gain        119
capital_loss         92
hours_per_week       94
native_country       41
SalaryGT50K           2
dtype: int64

In [ ]:
%%writefile {_transform_module_file}

# Features to be scaled to the z-score
_DENSE_FLOAT_FEATURE_KEYS = ['fnlwgt','age','capital_gain','capital_loss','hours_per_week']


# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
_VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
_OOV_SIZE = 10

# Features with string data types that will be converted to indices
_VOCAB_FEATURE_KEYS = ['workclass','education','marital_status','occupation','relationship','race','sex','native_country' ]

# Features with int data type that will be kept as is
_CATEGORICAL_FEATURE_KEYS = ['education_num']

# Feature to predict
_Label_KEY = 'SalaryGT50K'

def _transformed_name(key):
    return key

Writing transform_trainer.py


In [ ]:
%%writefile -a {_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}
    

    for key in _DENSE_FLOAT_FEATURE_KEYS:
        # Scale these features to the z-score.
        outputs[_transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))
    for key in _VOCAB_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(_fill_in_missing(inputs[key]), 
                                                                           top_k=_VOCAB_SIZE, 
                                                                           num_oov_buckets=_OOV_SIZE,
                                                                           vocab_filename=key)
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])

    
    outputs[_transformed_name(_Label_KEY)] = _fill_in_missing(inputs[_Label_KEY])


    return outputs


def _fill_in_missing(x):
    
    if not isinstance(x, tf.sparse.SparseTensor):
        return x

    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Appending to transform_trainer.py


In [ ]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    force_tf_compat_v1=True,  
    module_file=os.path.abspath(_transform_module_file))

In [ ]:
context.run(transform, enable_cache=False)

Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /content/tfx/Transform/transform_graph/6/.temp_path/tftransform_tmp/074334f185214731a6c1dc49607e4b18/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /content/tfx/Transform/transform_graph/6/.temp_path/tftransform_tmp/b556f4cb245540dd81d33c2c8e

INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /content/tfx/Transform/transform_graph/6/.temp_path/tftransform_tmp/86a87106eeee4d2b98b71b3665e937c8/assets
INFO:tensorflow:SavedModel written to: /content/tfx/Transform/transform_graph/6/.temp_path/tftransform_tmp/86a87106eeee4d2b98b71b3665e937c8/saved_model.pb
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
value

ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "/content/tfx/Transform/transform_graph/6"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/content/tfx/Transform/transformed_examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"eval\", \"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "/content/tfx/Transform/updated_analyzer_cache/6"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "/content/tfx/Transform/pre_transform_schema/6"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
      

In [ ]:
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []

    ### START CODE HERE
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = MessageToDict(example)
        
        # append to the records list
        records.append(example_dict)
        
    ### END CODE HERE
    return records

try:
    # Get the URI of the output artifact representing the transformed examples
    train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
    
except IndexError:
    print("context.run() was no-op")
    train_uri = os.path.join(transform_graph_uri, 'train')
    
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]


transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

sample_records_xf = get_records(transformed_dataset, 3)


pp.pprint(sample_records_xf)

[{'features': {'feature': {'SalaryGT50K': {'int64List': {'value': ['0']}},
                           'age': {'floatList': {'value': [0.83017725]}},
                           'capital_gain': {'floatList': {'value': [-0.14532618]}},
                           'capital_loss': {'floatList': {'value': [-0.21474226]}},
                           'education': {'int64List': {'value': ['2']}},
                           'education_num': {'int64List': {'value': ['13']}},
                           'fnlwgt': {'floatList': {'value': [-1.0109857]}},
                           'hours_per_week': {'floatList': {'value': [-2.2176924]}},
                           'marital_status': {'int64List': {'value': ['0']}},
                           'native_country': {'int64List': {'value': ['0']}},
                           'occupation': {'int64List': {'value': ['1']}},
                           'race': {'int64List': {'value': ['0']}},
                           'relationship': {'int64List': {'value': ['0']

In [ ]:
pip install -q tensorflow_cloud

     |████████████████████████████████| 92 kB 334 kB/s 


In [ ]:
%%writefile {_model_trainer_module_file}

"""WideDeep Classifier."""
import functools
import absl
import os
from typing import List, Text

import kerastuner
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_transform as tft

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx.components.tuner.component import TunerFnResult
from tfx_bsl.tfxio import dataset_options
from tensorflow_cloud import CloudTuner

LABEL_KEY = 'SalaryGT50K'
TRAIN_BATCH_SIZE = 128
EVAL_BATCH_SIZE = 512
BUCKET_SIZE = 30

# Define feature columns(Including feature engineered ones )
# These are the features which come from the TF Data pipeline
def create_feature_cols():
    #Keras format features
    k_age = tf.keras.Input(name='age', shape=(1,), dtype=tf.float32, sparse=False)
    k_capital_gain  = tf.keras.Input(name='capital_gain', shape=(1,), dtype=tf.float32, sparse=False)
    k_capital_loss  = tf.keras.Input(name='capital_loss', shape=(1,), dtype=tf.float32, sparse=False)
    k_education  = tf.keras.Input(name='education', shape=(1,), dtype=tf.int64, sparse=False)
    k_education_num  = tf.keras.Input(name='education_num', shape=(1,), dtype=tf.int64, sparse=False)
    k_fnlwgt = tf.keras.Input(name='fnlwgt', shape=(1,), dtype=tf.float32, sparse=False)
    k_hours_per_week  = tf.keras.Input(name='hours_per_week', shape=(1,), dtype=tf.float32, sparse=False)
    k_marital_status  = tf.keras.Input(name='marital_status', shape=(1,), dtype=tf.int64, sparse=False)
    k_native_country  = tf.keras.Input(name='native_country', shape=(1,), dtype=tf.int64, sparse=False)
    k_occupation  = tf.keras.Input(name='occupation', shape=(1,), dtype=tf.int64, sparse=False)
    k_race = tf.keras.Input(name='race', shape=(1,), dtype=tf.int64, sparse=False)
    k_relationship = tf.keras.Input(name='relationship', shape=(1,), dtype=tf.int64, sparse=False)
    k_sex = tf.keras.Input(name='sex', shape=(1,), dtype=tf.int64, sparse=False)
    k_workclass= tf.keras.Input(name='workclass', shape=(1,), dtype=tf.int64, sparse=False)


    keras_dict_input = {'age':k_age,'capital_gain':k_capital_gain,'capital_loss':k_capital_loss,'education':k_education,
                        'education_num':k_education_num,'fnlwgt':k_fnlwgt,'hours_per_week':k_hours_per_week,'marital_status':k_marital_status,
                        'native_country':k_native_country,'occupation':k_occupation,'race':k_race,'relationship':k_relationship,'sex':k_sex,
                        'workclass':k_workclass
                        }

    return({'K' : keras_dict_input})

def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def get_serving_signature(_mod, tfo):
    _mod.tft_layer_inference = tfo.transform_features_layer()
    _mod.raw_feature_spec = tfo.raw_feature_spec()

    @tf.function
    def serving(age,capital_gain,capital_loss,education,education_num,fnlwgt,hours_per_week,marital_status,native_country,occupation,race,
                relationship,sex,workclass):
        #Params coming in request
        features = {
            'age': age,
            'capital_gain': capital_gain,
            'capital_loss': capital_loss,
            'education': education,
            'education_num': education_num,
            'fnlwgt': fnlwgt,
            'hours_per_week': hours_per_week,
            'marital_status': marital_status,
            'native_country':native_country,
            'occupation':occupation,
            'race':race,
            'relationship':relationship,
            'sex':sex,
            'workclass':workclass
        }

        #Generate raw feature spec
        raw_feature_spec = _mod.raw_feature_spec
        # print(raw_feature_spec)
        
        #Add TFT transformations
        raw_features = {}
        for key, val in features.items():
            if key not in raw_feature_spec:
                continue
            if isinstance(raw_feature_spec[key], tf.io.VarLenFeature):
                raw_features[key] = tf.RaggedTensor.from_tensor(tf.expand_dims(val, -1)).to_sparse()
                continue
            raw_features[key] = val
            
            ##Apply INT32 to INT64 conversion because TFT accepts INT64 and we have taken all INTs as INT32
            if raw_feature_spec[key].dtype == tf.int64:
               raw_features[key] = tf.cast(raw_features[key], tf.int64)
        
        #Get all features from TFT
        tft_features = _mod.tft_layer_inference(raw_features)

        ## Predict
        ##IF THERE IS AN ERROR IN NUMBER OF PARAMS PASSED HERE OR DATA TYPE THEN IT GIVES ERROR, "COULDN'T COMPUTE OUTPUT TENSOR"
        predictions = _mod(tft_features)
        return predictions
    
    return serving

def tuner_fn(fn_args: TrainerFnArgs) -> TunerFnResult:
  """Build the tuner using the KerasTuner API.
  Args:
    fn_args: Holds args as name/value pairs.
      - working_dir: working dir for tuning.
      - train_files: List of file paths containing training tf.Example data.
      - eval_files: List of file paths containing eval tf.Example data.
      - train_steps: number of train steps.
      - eval_steps: number of eval steps.
      - schema_path: optional schema of the input data.
      - transform_graph_path: optional transform graph produced by TFT.
  Returns:
    A namedtuple contains the following:
      - tuner: A BaseTuner that will be used for tuning.
      - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                    model , e.g., the training and validation dataset. Required
                    args depend on the above tuner's implementation.
  """
  transform_graph = tft.TFTransformOutput(fn_args.transform_graph_path)
  
  # Construct a build_keras_model_fn that just takes hyperparams from get_hyperparameters as input.
  build_keras_model_fn = functools.partial(create_keras_model, tf_transform_output=transform_graph)  

  # BayesianOptimization is a subclass of kerastuner.Tuner which inherits from BaseTuner.    
  tuner = kerastuner.Hyperband(
      build_keras_model_fn,
      max_epochs=10,
      hyperparameters=_get_hyperparameters(),
      # New entries allowed for n_units hyperparameter construction conditional on n_layers selected.
#       allow_new_entries=True,
#       tune_new_entries=True,
      objective=kerastuner.Objective('val_accuracy', 'min'),
      directory=fn_args.working_dir,
      project_name='salary_tuning')
  
#   tuner = CloudTuner(
#       build_keras_model_fn,
#       # max_epochs=10,
#       hyperparameters=_get_hyperparameters(),
#       # New entries allowed for n_units hyperparameter construction conditional on n_layers selected.
# #       allow_new_entries=True,
# #       tune_new_entries=True,
#       objective=kerastuner.Objective('val_rmse', 'min'),
#       directory=fn_args.working_dir,
#       project_name='taxi_tuning',
#       project_id='hasanrafiq-test-331814',
#       region='us-central1'
#     )  
    
  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      transform_graph,
      batch_size=TRAIN_BATCH_SIZE)

  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      transform_graph,
      batch_size=EVAL_BATCH_SIZE)

  return TunerFnResult(
      tuner=tuner,
      fit_kwargs={
          'x': train_dataset,
          'validation_data': eval_dataset,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      })

def _input_fn(file_pattern: List[Text],
              data_accessor: DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.
  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch
  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)
    
  return dataset

def _get_hyperparameters() -> kerastuner.HyperParameters:
  """Returns hyperparameters for building Keras model."""
  hp = kerastuner.HyperParameters()
  hp.Float('learning_rate', 0.0001, 0.01, default=0.001)
  hp.Int('hidden_layers', 1, 1, step=1, default=1)
  return hp  

def create_keras_model(hparams: kerastuner.HyperParameters,
                       tf_transform_output: tft.TFTransformOutput):

    feature_cols = create_feature_cols()
    cat_fields = {}

    METRICS = [
        keras.metrics.Accuracy(name='accuracy')
    ]

    #Input layers
    input_feats = []
    for inp in feature_cols['K'].keys():
      input_feats.append(feature_cols['K'][inp])

    ##Input processing
    ##https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
    ##https://github.com/tensorflow/community/blob/master/rfcs/20191212-keras-categorical-inputs.md

    ##Handle categorical attributes( One-hot encoding by taking input as integer )
    for field in ['workclass','education','marital_status','occupation','relationship','race','sex','native_country']:
        cat_fields[field] = tf.keras.layers.CategoryEncoding(num_tokens=tf_transform_output.vocabulary_size_by_name(vocab_filename=field) + 1)(feature_cols['K'][field])

    ##Handle Bucket attributes( One-hot encoding by taking input as integer )
    for field in ['education_num']:
        cat_fields[field] = tf.keras.layers.CategoryEncoding(num_tokens=BUCKET_SIZE + 1)(feature_cols['K'][field])

    ###Create MODEL
    ####Concatenate all features( Numerical input )
    #'fnlwgt','age','capital_gain','capital_loss','hours_per_week'
    x_input_numeric = tf.keras.layers.concatenate([
                    feature_cols['K']['fnlwgt'], feature_cols['K']['age'],
                    feature_cols['K']['capital_gain'], feature_cols['K']['capital_loss'],
                    feature_cols['K']['hours_per_week']])

    #DEEP - This Dense layer connects to input layer - Numeric Data
    x_numeric = tf.keras.layers.BatchNormalization()(x_input_numeric)

    ####Concatenate all Categorical features( Categorical converted )
    x_categ = tf.keras.layers.concatenate([cat_fields[f_] for f_ in cat_fields.keys()]
    )
    
    ####Concatenate both Wide and Deep layers
    x = tf.keras.layers.concatenate([x_categ, x_numeric])

    for l_ in range(int(hparams.get('hidden_layers'))):
        x = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform", activity_regularizer=tf.keras.regularizers.l2(0.00001))(x)
        x = tf.keras.layers.BatchNormalization()(x)

    #Final Layer
    out = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(input_feats, out)

    #Set optimizer
    opt = tf.keras.optimizers.Adam(lr = hparams.get('learning_rate'))

    #Compile model
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  optimizer=opt, metrics = METRICS)

    #Print Summary
    print(model.summary())
    return model

def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.
  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files, 
      fn_args.data_accessor, 
      tf_transform_output, 
      TRAIN_BATCH_SIZE)

  eval_dataset = _input_fn(
      fn_args.eval_files, 
      fn_args.data_accessor,
      tf_transform_output, 
      EVAL_BATCH_SIZE)

  if fn_args.hyperparameters:
     hparams = kerastuner.HyperParameters.from_config(fn_args.hyperparameters)
  else:
    # This is a shown case when hyperparameters is decided and Tuner is removed
    # from the pipeline. User can also inline the hyperparameters directly in
    # _build_keras_model.
    hparams = _get_hyperparameters()
    absl.logging.info('HyperParameters for training: %s' % hparams.get_config())

  #  Distribute training over multiple replicas on the same machine.
  mirrored_strategy = tf.distribute.MirroredStrategy()
  with mirrored_strategy.scope():
    model = create_keras_model(
        hparams=hparams,
        tf_transform_output=tf_transform_output
        )

    #Add callbacks
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=5, min_lr=0.001, verbose = 1)

    model.fit(
      train_dataset,
      epochs=fn_args.custom_config['epochs'],
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True, verbose=True), 
                   reduce_lr]
      # callbacks=[tensorboard_callback]
      )

    #get signatures for serving_default -> Used for prediction via REST API
    serving = get_serving_signature(model, tf_transform_output).get_concrete_function(age=tf.TensorSpec([None,1], dtype= tf.int64, name='age'), 
                                        capital_gain=tf.TensorSpec([None,1], dtype= tf.int64, name='capital_gain'),
                                        capital_loss=tf.TensorSpec([None,1], dtype= tf.int64, name='capital_loss'), 
                                        education=tf.TensorSpec([None,1], dtype= tf.string, name='education'),
                                        education_num=tf.TensorSpec([None,1], dtype= tf.int64, name='education_num'), 
                                        fnlwgt=tf.TensorSpec([None,1], dtype= tf.int64, name='fnlwgt'),
                                        hours_per_week=tf.TensorSpec([None,1], dtype= tf.int64, name='hours_per_week'),
                                        marital_status=tf.TensorSpec([None,1], dtype= tf.string, name='marital_status'),
                                        native_country=tf.TensorSpec([None,1], dtype= tf.string, name='native_country'),
                                        occupation=tf.TensorSpec([None,1], dtype= tf.string, name='occupation'),
                                        race=tf.TensorSpec([None,1], dtype= tf.string, name='race'),
                                        relationship=tf.TensorSpec([None,1], dtype= tf.string, name='relationship'),
                                        sex=tf.TensorSpec([None,1], dtype= tf.string, name='sex'),
                                        workclass=tf.TensorSpec([None,1], dtype= tf.string, name='workclass')
                                        )

    model.save(fn_args.serving_model_dir, save_format='tf', signatures=serving)
        
  # model.save(fn_args.serving_model_dir + "/1", save_format='tf', signatures=sig)
  print("Model saved on: " + fn_args.serving_model_dir)    

Overwriting model.py


In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(_model_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=100), ##goes as train_steps / steps_per_epoch
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=3), ##goes as eval_steps
    transform_graph=transform.outputs['transform_graph'],
    custom_config=({"epochs": 20})
    )
context.run(trainer)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 workclass (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 education (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 marital_status (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 occupation (InputLayer)        [(None, 1)]          0           []                      

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


100/100 [==============================] - 5s 21ms/step - loss: 0.4688 - accuracy: 0.0011 - val_loss: 0.4287 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/20
100/100 [==============================] - 1s 9ms/step - loss: 0.3762 - accuracy: 0.0032 - val_loss: 0.3643 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/20
100/100 [==============================] - 1s 7ms/step - loss: 0.3434 - accuracy: 0.0020 - val_loss: 0.3270 - val_accuracy: 0.0026 - lr: 0.0010
Epoch 4/20
100/100 [==============================] - 1s 9ms/step - loss: 0.3311 - accuracy: 0.0020 - val_loss: 0.3227 - val_accuracy: 0.0039 - lr: 0.0010
Epoch 5/20
100/100 [==============================] - 1s 8ms/step - loss: 0.3255 - accuracy: 6.2500e-04 - val_loss: 0.3029 - val_accuracy: 0.0020 - lr: 0.0010
Epoch 6/20
100/100 [==============================] - 1s 7ms/step - loss: 0.3150 - accuracy: 0.0012 - val_loss: 0.3487 - val_accuracy: 0.0059 - lr: 0.0010
Epoch 7/20
100/100 [==============================] - 1s 9ms/step - 

ExecutionResult(
    component_id: Trainer
    execution_id: 22
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 39
        type_id: 25
        uri: "/content/tfx/Trainer/model/22"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 40
        type_id: 26
        uri: "/content/tfx/Trainer/model_run/22"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
!saved_model_cli show --dir "tfx/Trainer/model/16/Format-Serving/" --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['age'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_age:0
    inputs['capital_gain'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_capital_gain:0
    inputs['capital_loss'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_capital_loss:0
    inputs['education'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 1)
        name: serving_default_educati

In [ ]:
saved_mod = tf.keras.models.load_model("tfx/Trainer/model/16/Format-Serving/")


f = saved_mod.signatures['serving_default']


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f97c839ec10> and <keras.engine.input_layer.InputLayer object at 0x7f97caad77d0>).


In [ ]:
f(age=tf.convert_to_tensor([50]), workclass = tf.convert_to_tensor([["State-gov"]]),fnlwgt=tf.convert_to_tensor([77516]),
  education=tf.convert_to_tensor([["Bachelors"]]),education_num=tf.convert_to_tensor([13]),marital_status=tf.convert_to_tensor([["Never-married"]]),
  occupation=tf.convert_to_tensor([["Adm-clerical"]]),relationship=tf.convert_to_tensor([["Husband"]]),race=tf.convert_to_tensor([["White"]]),
  sex=tf.convert_to_tensor([["Male"]]),capital_gain=tf.convert_to_tensor([0]),capital_loss=tf.convert_to_tensor([0]),hours_per_week=tf.convert_to_tensor([40]), 
  native_country=tf.convert_to_tensor([["United-States"]]))
                                                                                                                                                    
  

InvalidArgumentError: ignored

In [ ]:
train_df.head(2)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,SalaryGT50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0


((),
 {'capital_gain': TensorSpec(shape=(None, 1), dtype=tf.int64, name='capital_gain'),
  'fnlwgt': TensorSpec(shape=(None, 1), dtype=tf.int64, name='fnlwgt'),
  'marital_status': TensorSpec(shape=(None, 1), dtype=tf.string, name='marital_status'),
  'education_num': TensorSpec(shape=(None, 1), dtype=tf.int64, name='education_num'),
  'age': TensorSpec(shape=(None, 1), dtype=tf.int64, name='age'),
  'race': TensorSpec(shape=(None, 1), dtype=tf.string, name='race'),
  'workclass': TensorSpec(shape=(None, 1), dtype=tf.string, name='workclass'),
  'occupation': TensorSpec(shape=(None, 1), dtype=tf.string, name='occupation'),
  'hours_per_week': TensorSpec(shape=(None, 1), dtype=tf.int64, name='hours_per_week'),
  'relationship': TensorSpec(shape=(None, 1), dtype=tf.string, name='relationship'),
  'sex': TensorSpec(shape=(None, 1), dtype=tf.string, name='sex'),
  'education': TensorSpec(shape=(None, 1), dtype=tf.string, name='education'),
  'native_country': TensorSpec(shape=(None, 1), dt

In [ ]:
saved_mod = tf.keras.models.load_model("tfx/Trainer/model/16/Format-Serving/")


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f97b811f4d0> and <keras.engine.input_layer.InputLayer object at 0x7f97b87040d0>).


In [ ]:
saved_mod.predict_function()